## Set up and import 

In [18]:
import io
import os
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv(f'{os.path.dirname(os.getcwd())}/data/jigsaw_toxic/train.csv')
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


## Config

In [29]:
n = 1000
toxic = [df[df['toxic']==1]['comment_text'][i:i+1].to_string(index=False) for i in range(0,n)]
non_toxic = [df[df['toxic']==0]['comment_text'][i:i+1].to_string(index=False) for i in range(0,n)]

In [24]:
model_name = "eleuther-pythia2.8b-hh-sft"
layer = -1


## Load hidden states for base, sft and dpo

In [30]:
x = []
y = []
for idx, model_name in enumerate(["pythia2.8b","eleuther-pythia2.8b-sft","eleuther-pythia2.8b-dpo"]):
  model_SAVEFOLD0 = f"{os.path.dirname(os.getcwd())}/outputs/{model_name}"
  model_SAVEFOLD = f"{model_SAVEFOLD0}/layer{layer}/"

  toxic_f = f"{SAVEFOLD}toxic_hs.npy"
  non_toxic_f = f"{SAVEFOLD}non_toxic_hs.npy"
  toxic_hs = np.load(toxic_f, mmap_mode = 'r')
  non_toxic_hs = np.load(non_toxic_f, mmap_mode = 'r')

  x.append(np.concatenate([non_toxic_hs, toxic_hs],0))
  y.append(np.concatenate([np.array(len(non_toxic)*[2*idx]), np.array(len(toxic)*[2*idx+1])],0))

x = np.concatenate([x[0], x[1], x[2]],0) 
y = np.concatenate([y[0], y[1], y[2]],0) 

In [33]:
x[0]

array([ 0.51742744,  0.92089754,  1.2923046 , ...,  0.40341443,
        2.0513492 , -0.16610062], dtype=float32)

In [34]:
x[2000]

array([ 0.51742744,  0.92089754,  1.2923046 , ...,  0.40341443,
        2.0513492 , -0.16610062], dtype=float32)

In [35]:
x[4000]

array([ 0.51742744,  0.92089754,  1.2923046 , ...,  0.40341443,
        2.0513492 , -0.16610062], dtype=float32)

In [36]:
direction = 0
base_sft_dot = np.empty([int(len(x)/3)])
base_sft_sim = np.empty([int(len(x)/3)])
base_dpo_dot = np.empty([int(len(x)/3)])
base_dpo_sim = np.empty([int(len(x)/3)])
sft_dpo_dot = np.empty([int(len(x)/3)])
sft_dpo_sim = np.empty([int(len(x)/3)])
for i in range(int(len(x)/3)):
    base_sft_dot[i] = np.dot(x[i], x[i+2*n])
    base_sft_sim[i] = base_sft_dot[i]/(np.linalg.norm(x[i])*np.linalg.norm(x[i+2*n]))
    # print(i, i+2*n)

for i in range(int(len(x)/3)):
    base_dpo_dot[i] = np.dot(x[i], x[i+4*n])
    base_dpo_sim[i] = base_dpo_dot[i]/(np.linalg.norm(x[i])*np.linalg.norm(x[i+4*n]))
    # print(i, i+4*n)

for i in range(int(len(x)/3)):
    sft_dpo_dot[i] = np.dot(x[i+2*n], x[i+4*n])
    sft_dpo_sim[i] = sft_dpo_dot[i]/(np.linalg.norm(x[i+2*n])*np.linalg.norm(x[i+4*n]))
    # print(i+2*n, i+4*n)

In [38]:
print(np.mean(base_sft_sim))
print(np.mean(base_dpo_sim))
print(np.mean(sft_dpo_sim))

0.9999999996432282
0.9999999996432282
0.9999999996432282


They're exactly the same